In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init()
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

from pydataset import data

import ard_fit_pipelines as fp
reload(fp.core)
reload(fp.transforms)
reload(fp.graph)
reload(fp)

logging.getLogger('ard_fit_pipelines').setLevel(logging.INFO)

2022-06-29 15:41:21,449:    INFO:      ard_logging: MainProcess 13289: Logging initialized.
2022-06-29 15:41:21,449:    INFO:         __main__: MainProcess 13289: Hello from notebook


In [2]:
df = data('diamonds')
ds = fp.PandasDataset(df)

In [3]:
ds.to_dataframe()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [4]:
dsc = fp.DatasetCollection({'__data__': ds})

In [5]:
dsc.to_dataframe()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [7]:
iddy = fp.Identity()
diddy = fp.Pipeline('foo')

In [8]:
diddy

Pipeline(dataset_name='foo', transforms=[])

In [12]:
iddy.fit(df)
iddy.fit(ds)
iddy.fit(dsc)

FitIdentity(<53940 rows of fitting data>)

In [11]:
diddy.fit(df)
diddy.fit(ds)
diddy.fit(dsc)

UnknownDatasetError: "Asked for a Dataset named 'foo' but this DatasetCollection only has: ['__data__']"

In [13]:
diddy.fit(fp.DatasetCollection({'foo': ds}))

FitPipeline(dataset_name='foo', transforms=[], <53940 rows of fitting data>)

In [14]:
cols = ["carat", "x", "y", "z", "depth", "table"]
pipeline_chain = (
    fp.Pipeline()
    .pipe(["carat"], np.log1p)
    .winsorize(cols, limit=0.05)
    .z_score(cols)
    .impute_constant(cols, 0.0)
    .clip(cols, upper=2, lower=-2)
)

In [15]:
pipeline_chain

Pipeline(dataset_name='__data__', transforms=[Pipe(cols=['carat'], apply_fun=<ufunc 'log1p'>), Winsorize(cols=['carat', 'x', 'y', 'z', 'depth', 'table'], limit=0.05), ZScore(cols=['carat', 'x', 'y', 'z', 'depth', 'table'], w_col=None), ImputeConstant(cols=['carat', 'x', 'y', 'z', 'depth', 'table'], value=0.0), Clip(cols=['carat', 'x', 'y', 'z', 'depth', 'table'], upper=2, lower=-2)])